In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 7.9 MB/s 
     |████████████████████████████████| 453 kB 89.7 MB/s 


In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 4.7 MB 18.1 MB/s 
     |████████████████████████████████| 1.3 MB 70.1 MB/s 
     |████████████████████████████████| 120 kB 52.8 MB/s 
     |████████████████████████████████| 6.6 MB 66.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=dbeb38ec983ecfc601293cd0cece243fa59f19dd427a762532522115716d5c78
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7fecf8bf1690>)

In [ ]:
train_data = pd.read_table('./ratings_train.txt')
test_data = pd.read_table('./ratings_test.txt')

In [ ]:
train_data.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
train_data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


전처리 후 테스트용 샘플의 개수 : 48852


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    status = ""
    for i in stopwords_removed_sentence:
      status += i + " "
    X_train.append(status)

In [ ]:
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    status = ""
    for i in stopwords_removed_sentence:
      status += i + " "
    X_test.append(status)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
X_train = np.delete(X_train, drop_train, axis=0)
print(len(X_train))

In [ ]:
# tokenizer = Tokenizer(vocab_size) 
# tokenizer.fit_on_texts(X_train)
# X_train = tokenizer.texts_to_sequences(X_train)
# X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# max_len = 50

In [ ]:
# X_train = pad_sequences(X_train, maxlen=max_len)
# X_test = pad_sequences(X_test, maxlen=max_len)

# Doc2Vec https://wikidocs.net/155356

In [ ]:
import pandas as pd
from konlpy.tag import Mecab, Okt
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm
import urllib.request
from gensim.models import doc2vec
import numpy as np

In [ ]:
train_data['label'].value_counts()

0    73342
1    72840
Name: label, dtype: int64

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f9967868990>)

In [ ]:
train_data = pd.read_table('./ratings_train.txt')
test_data = pd.read_table('./ratings_test.txt')

In [ ]:
train_data.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 48852


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
okt = Okt()

tagged_corpus_list = []

for index, row in tqdm(train_data.iterrows(), total=len(train_data)):
  text = row['document']
  tag = row['document']
  stopwords_removed_sentence = [word for word in okt.morphs(text) if not word in stopwords]
  tagged_corpus_list.append(TaggedDocument(tags=[tag], words=stopwords_removed_sentence))

print('\n문서의 수 :', len(tagged_corpus_list))

100%|██████████| 146182/146182 [06:52<00:00, 354.59it/s]


문서의 수 : 146182


In [ ]:
# model = doc2vec.Doc2Vec(vector_size=100, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# # Vocabulary 빌드
# model.build_vocab(tagged_corpus_list)
# print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# # Doc2Vec 학습
# model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# # 모델 저장
# model.save('./similarity_detection.doc2vec')

In [ ]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

# Vocabulary 빌드
model.build_vocab(tagged_corpus_list)
print(f"Tag Size: {len(model.docvecs.doctags.keys())}", end=' / ')

# Doc2Vec 학습
model.train(tagged_corpus_list, total_examples=model.corpus_count, epochs=50)

# 모델 저장
model.save('./similarity_detection_v=300_okt.doc2vec')

Tag Size: 146182 / 

In [ ]:
test_corpus_list = []

for index, row in tqdm(test_data.head(30).iterrows(), total=len(test_data.head(30))):
  text = row['document']
  tag = row['document']
  stopwords_removed_sentence = [word for word in .morphs(text) if not word in stopwords]
  test_corpus_list.append(TaggedDocument(tags=[tag], words=stopwords_removed_sentence))

In [ ]:
train = np.array(train_data['document'])
test = np.array(test_data.head(30)['document'])
text_score = [0 for i in range(len(test))]
text_score_sentence = [0 for i in range(len(test))]

In [ ]:
for i in range(len(test)):
  for j in tqdm(range(len(train))):
    score = model.docvecs.similarity_unseen_docs(model, test[i], train[j])
    if score > text_score[i]:
      text_score[i] = max(text_score[i] , score)
      text_score_sentence[i] = train[j]
    


100%|██████████| 146182/146182 [02:43<00:00, 894.29it/s]


In [ ]:
text_score_sentence

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '각자의 아버지를 돌아보라',
 'OST가 아깝다..',
 '재미없었어',
 '도대체 무슨영화인지 평점만으로는 가늠하기힘들었다',
 '전문가평점좀없앴으면;',
 '할말없음...',
 '오글거려못보겟다 연기다들너무못하는듯',
 '아.... 탁.탁.탁.',
 '앜ㅋㅋㅋ 서리한이라닠ㅋㅋㅋㅋㅋ',
 '은행 강도 영화라고 하기에는 너무 부족하고 형제애를 그렸다고하기에도 어이없고 사랑얘기도 아니고 기껏 경찰들 시선을 다른데로 돌려놓고는 정작 경찰이 출동하니 어영부영 시간끌고있는 어이없는 상황 보다 답답해 미침',
 '평점깎기.',
 '재밌게 본 영화 연기 쩔어',
 '민주화 시대의 억눌린 영혼의 관음적 욕구 분출신고',
 '좋아하는 코믹영화 중 하나',
 '보지도 않고 주는건 아닌것 같은데요 시사회에서 봤을수도 있지만 해외에서 보고 오신분들도 있을겁니다 그리고 아무리 뮤지컬 실황이라도 영화로 나왔으면 그에 걸맞는 관람요소가 있어야 하지 않나요? 제가 보기엔 뮤지컬이라면 모를까 영화로서의 평점은 1점입니다',
 '이거재밌게봤던기억이',
 '오늘 OBS에서 하길래 봤더니만, 진짜 시간 더럽게 아깝더라',
 '김치워리어는 이거 보고 본받아라 ㅉㅉ',
 '마음이 따뜻해지는 영화였습니다.',
 'OO']

In [ ]:
test[10:]

array(['한국독립영화의 한계 그렇게 아버지가 된다와 비교됨',
       '청춘은 아름답다 그 아름다움은 이성을 흔들어 놓는다 찰나의 아름다움을 잘 포착한 섬세하고 아름다운 수채화같은 퀴어영화이다',
       '눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다',
       '스토리 연출 연기 비주얼 등 영화의 기본 조차 안된 영화에 무슨 평을 해 이런 영화 찍고도 김문옥 감독은 내가 영화 경력이 몇인데 조무래기들이 내 영화를 평론해 같은 마인드에 빠져있겠지',
       '소위 ㅈ문가라는 평점은 뭐냐', '최고', '발연기 도저히 못보겠다 진짜 이렇게 연기를 못할거라곤 상상도 못했네',
       '나이스', '별 재미도없는거 우려먹어  챔프에서 방송 몇번했더라  ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
       '일의 금요일 나이트메어시리즈와 함께 가장 많은 시리즈를 양산해냈던 헬레이저 시리즈의 첫편 작가의 상상력이 돋보이는 작품이며 갈고리로 사지찢는 고어씬은 지금보더라도 상당히 잔인하고 충격적이다',
       '나름 교훈돋기는 하지만 어쩔수없이 저평점 받을수밖에 없는 저질섹스코미디', '꽤 재밌게 본 영화였다',
       '민주화 시대의 억눌린 영혼의 관음적인 욕구 분출인상적이다',
       '일본 천황이 미국으로부터 받은 면죄부의 긴박한 과정을 루즈하고 지저분하게 늘어놓았다',
       '괜히 나올 때 어미 알 건드려서 긁어 부스름 만들었다는 분이 저 아래 보이던데 영화 제대로 안 봤네 알이 딱 까지면서 새끼가 나오려 했음 그냥 가면 그 놈 한테 당했을 것임 한 놈 두 놈 막 나올 게 뻔했으니 작살낼 수 밖에 없었다',
       '번은 봤네요어쩜 이렇게 잘만들었을까', '실화라서더욱아름답고찡하네요많이울었어요벌써년이란시간이흘렀네요',
       '이건뭐 노답이네 년대 어린이 영화좀 보고 본받아라', '지금까지 본 영화중 마음이 가장 따뜻해지는 영화',
       '너무너무 재밌다'], dtype=object)

In [ ]:
# text_score = [0 for i in range(len(test_corpus_list))]
# text_score_sentence = [0 for i in range(len(test_corpus_list))]

In [ ]:
# for i in range(len(test_corpus_list)):
#   for j in tqdm(range(len(tagged_corpus_list))):
#     score = model.docvecs.similarity_unseen_docs(model, test_corpus_list[i], tagged_corpus_list[j])
#     if score > text_score[i]:
#       text_score[i] = max(text_score[i] , score)
#       text_score_sentence[i] = np.array(train_data['document'])[j]

# TF, TF-IDF -> 램 부족으로 유기

In [ ]:
tfvector = CountVectorizer()
tfvector.fit(X_train)
# 코퍼스로부터 각 단어의 빈도수를 기록

# 각 단어와 맵핑된 인덱스 출력
# print(vector.vocabulary_)

CountVectorizer()

In [ ]:
tfvector.fit_transform(X_train)

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)
x_tfidf = tfidf.fit_transform(X_train)

In [ ]:
len(X_train)

195034

In [ ]:
x_tfidf.A

In [ ]:
tfidf.transform(X_train).toarray()

# 가짜 리뷰(비슷한 리뷰 자카드 유사도 활용) ㄹㅇ 프로토타입



In [ ]:
text = ["와 영화 개노잼이네 진짜 망할듯",
        "시간 가는 줄 모르고 재밌게 봤습니다.",
        "ㄹㅇ 개노잼"]

In [ ]:
text_processing = []
for i in text:
    tokenized_sentence = okt.morphs(i, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    status = ""
    for i in stopwords_removed_sentence:
      status += i + " "
    text_processing.append(status)

NameError: ignored

In [ ]:
text_processing = tokenizer.texts_to_sequences(text_processing)

In [ ]:
text_score = [0 for i in range(len(text))]

for i in range(len(text_processing)):
  for j in X_train:
    union = set(text_processing[i]).union(set(j))
    intersection = set(text_processing[i]).intersection(set(j))
    text_score[i] = max(text_score[i], len(intersection) / len(union))

In [ ]:
text_score

# sentenceBERT 실험

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7f7ab91a16d0>)

In [ ]:
train_data = pd.read_table('./ratings_train.txt')
test_data = pd.read_table('./ratings_test.txt')

In [ ]:
train_data.drop_duplicates(subset=['document'], inplace=True)

In [ ]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [ ]:
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


전처리 후 테스트용 샘플의 개수 : 48852


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/967k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/394 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
train_embedding = model.encode(np.array(train_data['document']))

In [ ]:
train_embedding.shape

(146182, 768)

In [ ]:
test_embedding = model.encode(np.array(test_data['document']))

In [ ]:
test_embedding.shape

(48852, 768)

In [ ]:
def cosine(a,b):
  return np.dot(a,b) / np.sqrt(np.dot(a, a) * np.dot(b, b))

In [ ]:
cosine(test_embedding[0], train_embedding[0])

0.13432628

In [ ]:
length = 1000

train = np.array(train_data['document'])
test = np.array(test_data['document'])[:length]
test_score = np.array([[0,0,test[i]] for i in range(length)])

In [ ]:
test_score[i,:]

array(['0', '0', '굳 ㅋ'], dtype='<U140')

In [ ]:
for i in tqdm(range(len(test))):
  for j in range(len(train)):
    score = cosine(train_embedding[j], test_embedding[i])
    if score > float(test_score[i,0]):
      test_score[i,0] = score
      test_score[i,1] = train[j]

100%|██████████| 1000/1000 [20:21<00:00,  1.22s/it]


In [ ]:
test_score[test_score[:,0].astype('float64') > 0.9]

array([['0.93384105', '굳ㅋ', '굳 ㅋ'],
       ['0.9110824', '최고의 쓰레기', '진정한 쓰레기'],
       ['1.0', '최고', '최고'],
       ['1.0', '나이스', '나이스'],
       ['0.9421702', '의외로 재밌게 본 영화다.', '꽤 재밌게 본 영화였다'],
       ['0.9195655', '민주화 시대의 억눌린 영혼의 관음적 욕구 분출신고',
        '민주화 시대의 억눌린 영혼의 관음적인 욕구 분출인상적이다'],
       ['0.96821856', '너무 재밌다', '너무너무 재밌다'],
       ['1.0', '감동적이다', '감동적이다'],
       ['0.94548047', '난 재밌는데 평점 왜케 낮음', '난 재밌던데 평점 왜케 낮지 '],
       ['0.93806803', '재밌습니다 연기도좋고 내용도좋고',
        '재밌다 내용도 신선하고 의미도있으며 연기도 좋고 영상도좋다'],
       ['1.0', '볼만함', '볼만함'],
       ['0.9266063', '잼없음', '완전 잼없음'],
       ['0.94674283', '시리즈중에 단연 최고!', '시리즈중에는 단연 최고'],
       ['0.9032344', '내가 본 영화중에 최악이었다', '내가 극장가서 본 영화중 제일 쓰레기 같았다'],
       ['1.0', '너무 좋았음', '너무 좋았음'],
       ['0.96135765', '너무 재밌게 잘 봤습니다', '정말 재밌게 잘봤습니다'],
       ['0.91165817', '간만에 통쾌하게 웃었네', '간만에 참 유쾌하게 웃은듯'],
       ['1.0', '굿', '굿'],
       ['0.97036093', '심심할때 보면 딱 좋은 영화인듯', '심심할때 보면 딱 좋은 영화'],
       ['0.95168734', '사랑이 무엇인지 알게 해주는 영화', '사랑이 무

In [ ]:
test_score[:,0] = test_score[:,0].astype('float64')
test_score[:,0]

array(['0.93384105', '0.75942147', '0.75616467', '0.67215717',
       '0.83094877', '0.9110824', '0.6060709', '0.7946528', '0.7416692',
       '0.6968709', '0.634077', '0.6855309', '0.7350456', '0.73914206',
       '0.57397145', '1.0', '0.7279476', '1.0', '0.7202091', '0.67276907',
       '0.68516487', '0.9421702', '0.9195655', '0.58657956', '0.6283043',
       '0.7324451', '0.7691063', '0.6937067', '0.8850936', '0.96821856',
       '0.83736014', '0.7695779', '0.79592454', '0.77103525', '0.5973081',
       '0.86648625', '0.5810734', '0.7326218', '0.7695719', '0.76740056',
       '0.6920596', '0.882871', '0.7045006', '0.56072974', '1.0',
       '0.49964938', '0.8043319', '0.7990222', '0.6418879', '0.83164084',
       '0.5576299', '0.64999866', '0.71902853', '0.88099027',
       '0.94548047', '0.6319773', '0.62687165', '0.93806803', '0.7069259',
       '0.7515321', '0.6895017', '0.68937266', '0.6271592', '0.77209973',
       '0.7670207', '0.88911', '0.80844', '0.8023861', '0.72982424',
 

In [ ]:
txt1 = "완전 좋은 제품이네요 굿굿!!"
txt2 = "완전 좋은 제품이네요 굿굿ㅋㅋ"
emb1 = model.encode(txt1)
emb2 = model.encode(txt2)
print(cosine(emb1, emb2))

0.97496045


# 리뷰 요약 https://github.com/seujung/KoBART-summarization

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, PreTrainedTokenizer

from transformers import BartForConditionalGeneration

In [ ]:
# tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
# model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
text = """
늘 사용하는 제품입니다!!
휴지는 달라붙으면 갖다버리라는 할머님의 말씀
항상 기억하며 안달라붙는 제품 찾느라
휴지 유목민 생활을 하던중에
핫딜이 떠서 한번 구매했다가 계속 이제품만
사용하게된 케이스구요~
동영상에도 나와있듯 달라붙지 않아요
도톰하고 은은한 향이 나서 사용할때마다
기분 좋아지는 제품입니다!!
간혹 같은 회사 같은 제품인데도 그 질이
약간씩 다를때가 있는데~
다행히 이번 제품은 우수한 품질이네요
배송도 빠르게 잘 왔구요~
다만 비닐포장 단 하나로 오는데
이게 장마철에 배송되면 자칫 습기가
먹을수도 있다고 생각됩니다
그런점은 판매자분의 개선사항이 아닐까 싶네요
제품에는 전혀 불만없는 아주 만점이에요^^
"""

text = text.replace('\n', ' ')

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),
                             length_penalty = 0.75,
                             num_beams=8,  
                             max_length=128,  
                             eos_token_id=1)
output = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
print(output)

휴지 유목민 생활을 하던 중 핫딜이 떠서 한번 구매했다가 계속 이제품만 사용하게된 케이스로 도톰하고 은은한 향이 나서 사용할때마다 기분 좋아지는 제품인데 이번 제품은 우수한 품질이네요 배송도 빠르게 잘 왔구요~ 다만 비닐포장 단 하나로 오는데 이게 장마철에 배송되면 자칫 습기가 먹을수도 있다고 생각됩니다 그런점은 판매자분의 개선사항이 아닐까 싶네요 제품에는 전혀 불만없는 아주 만점이에요 


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, PreTrainedTokenizer
from transformers import BartForConditionalGeneration

def kobart_summary(text):
  tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
  model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
  text = text.replace('\n', ' ')

  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  summary_ids = model.generate(torch.tensor([input_ids]),
                              length_penalty = 0.5,
                              num_beams=8,  
                              max_length=128,  
                              eos_token_id=1)
  output = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  print(output)
  return output


In [ ]:
# result
"""
휴지 유목민 생활을 하던중 
핫딜이 떠서 한번 구매했다가
계속 이제품만 사용하게된 케이스로
 도톰하고 은은한 향이 나서 사용할때마다
  기분 좋아지는 제품인데 
  이번 제품은 우수한 품질이네요
   배송도 빠르게 잘 왔구요~
 다만 비닐포장 단 하나로 오는데
  이게 장마철에 배송되면 자칫 습기가
   먹을수도 있다고 생각됩니다 
그런점은 판매자분의 개선사항이 아닐까 싶네요 
제품에는 전혀 불만없는 아주 만점이에요
"""

In [ ]:
text = """
늘 사용하는 제품입니다!!
휴지는 달라붙으면 갖다버리라는 할머님의 말씀
항상 기억하며 안달라붙는 제품 찾느라
휴지 유목민 생활을 하던중에
핫딜이 떠서 한번 구매했다가 계속 이제품만
사용하게된 케이스구요~
동영상에도 나와있듯 달라붙지 않아요
도톰하고 은은한 향이 나서 사용할때마다
기분 좋아지는 제품입니다!!
간혹 같은 회사 같은 제품인데도 그 질이
약간씩 다를때가 있는데~
다행히 이번 제품은 우수한 품질이네요
배송도 빠르게 잘 왔구요~
다만 비닐포장 단 하나로 오는데
이게 장마철에 배송되면 자칫 습기가
먹을수도 있다고 생각됩니다
그런점은 판매자분의 개선사항이 아닐까 싶네요
제품에는 전혀 불만없는 아주 만점이에요^^
"""

text = text.replace('\n', ' ')

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),
                             length_penalty = 0.5, # 길이에 대한 패널티. 1보다 작을 수록 짧게, 클 수록 길게 출력
                             num_beams=8,  # 문장 생성시 다음 단어를 탐색하는 영역의 수
                             max_length=128,   #  요약문의 최대 길이
                             eos_token_id=1)
output = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
print(output)


휴지 유목민 생활을 하던중 핫딜이 떠서 한번 구매했다가 계속 이제품만 사용하게된 케이스로 도톰하고 은은한 향이 나서 사용할때마다 기분 좋아지는 제품입니다!!


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, PreTrainedTokenizer
from transformers import BartForConditionalGeneration

def kobart_summary(text):
  tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
  model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
  text = text.replace('\n', ' ')

  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  summary_ids = model.generate(torch.tensor([input_ids]),
                              length_penalty = 0.5,
                              num_beams=8,  
                              max_length=128,  
                              eos_token_id=1)
  output = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  print(output)
  return output

In [ ]:
text = """
늘 사용하는 제품입니다!!
휴지는 달라붙으면 갖다버리라는 할머님의 말씀
항상 기억하며 안달라붙는 제품 찾느라
휴지 유목민 생활을 하던중에
핫딜이 떠서 한번 구매했다가 계속 이제품만
사용하게된 케이스구요~
동영상에도 나와있듯 달라붙지 않아요
도톰하고 은은한 향이 나서 사용할때마다
기분 좋아지는 제품입니다!!
간혹 같은 회사 같은 제품인데도 그 질이
약간씩 다를때가 있는데~
다행히 이번 제품은 우수한 품질이네요
배송도 빠르게 잘 왔구요~
다만 비닐포장 단 하나로 오는데
이게 장마철에 배송되면 자칫 습기가
먹을수도 있다고 생각됩니다
그런점은 판매자분의 개선사항이 아닐까 싶네요
제품에는 전혀 불만없는 아주 만점이에요^^
"""

kobart_summary(text)

In [ ]:
kobart_summary(text)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


휴지 유목민 생활을 하던중 핫딜이 떠서 한번 구매했다가 계속 이제품만 사용하게된 케이스로 도톰하고 은은한 향이 나서 사용할때마다 기분 좋아지는 제품입니다!!


'휴지 유목민 생활을 하던중 핫딜이 떠서 한번 구매했다가 계속 이제품만 사용하게된 케이스로 도톰하고 은은한 향이 나서 사용할때마다 기분 좋아지는 제품입니다!!'

# KeyBert 키워드 추출

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
doc = """
드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
"향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
"""

In [ ]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('드론', 'Noun'), ('활용', 'Noun'), ('범위', 'Noun'), ('도', 'Josa'), ('점차', 'Noun'), ('확대', 'Noun'), ('되고', 'Verb'), ('있다', 'Adjective'), ('.', 'Punctuation')]
명사 추출 : 드론 활용 범위 점차 확대 최근 미세먼지 관리 드론 활용 서울시 미세먼지 계절 관리제 기간 지난달 개 월간 드론 측정 장치 달 미세먼지 집중 관리 실시 드론 산업 단지 사업 밀집 지역 미세먼지 배출 수치 점검 현장 모습 영상 영상 통해 미세먼지 방지 시설 제대로 가동 업체 무허가 시설 대한 단속 한층 전망 드론 활용 가장 적극 소방청 복합 재난 대응 차원 드론 관련 전문 인력 보강 어가 지난해 말 기준 소방청 보유 드론 총 드론 조종 자격증 소방대 경우 명 이 중 실기 평가 지도 자격증 드론 전문가 명도 배치 소방청 관계자 소방 드론 재난 현장 영상 정보 수집 산악 수난 사고 시 인명 수색 구조 활동 유독가스 폭발사고 시 대원 안전 확보 등 활용 며 향후 화재 진압 인명구조 등 드론 활용 위해 연구개발 고 말


In [ ]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
pos_txt = ""
neg_txt = ""
for i in np.array(train_data[train_data['label'] == 1]['document']):
  pos_txt += i
for i in np.array(train_data[train_data['label'] == 0]['document']):
  neg_txt += i

In [ ]:
print(len(pos_txt), len(neg_txt))

2579640 2680184


In [ ]:
okt = Okt()

tokenized_doc = okt.pos(pos_txt)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)

품사 태깅 10개만 출력 : [('흠', 'Noun'), ('...', 'Punctuation'), ('포스터', 'Noun'), ('보고', 'Noun'), ('초딩', 'Noun'), ('영화', 'Noun'), ('줄', 'Noun'), ('....', 'Punctuation'), ('오버', 'Noun'), ('연기', 'Noun')]
명사 추출 : 흠 포스터 보고 초딩 영화 줄 오버 연기 몬페 의 연기 영화 스파이더맨 커스틴 던스트 액션 재미 몇 영화 왜케 평점 꽤 볼 데 헐리우드 인피니트 짱 진짜 볼때 향수 자극 허진호 감성 절제 멜로 달인 로만 자꾸 그 사람 것 냥 매번 긴장 재밋음 사람 바스코 이기 락스 코 바비 이기 아이돌 그냥 안달 것 정말 깨알 캐스팅 질퍽 용구성 버무러진 깨알 일드 약탈 위 변명 놈 놈 절대 걸 나름 뜻 듯 그냥 학생 선생 영화 절대 건 절대 영화 수작 는걸 고추 재밋 뎅 센스 연출 캐스팅 향수 점 엄포스 위력 다시 한번 적 남 꽃 검사 연기 정말 완전 명품 드라마 점 왜 이 드라마 인생 최고 패션 대한 열정 안나 윈 투어 원작 정신 유령 이 영화 왜 평가 매력 영화 서리 이 정말 맘 또 또 방법 윤제문 배우 발견 탈 미소 머금 음악 조금 점 평점 점 데 리스 타르 용의 주인 누 근친상간 소설 속 제일 놈 자 메 니스 터 드라마 속 드래곤 용 이 제일 웃음 감독 토르 다크 월드 말 기본 선방 영화 사람 영혼 줄 수도 상사 잠시 동화 영화 가슴 시리 드라마 또 감동 그 자체 요전 충격 기분 푹 느낌 활 하나 일본인 상상력 정말 생각 백봉기 언제 사랑 가슴속 감정 영화 정말 최고 사람 이 다큐 보고 우리나라 현대 사의 단면 대해 깊이 생각 사죄 바로 잡기 위해 노력 말로 보도 연맹 그 민간인 학살 이정 일 줄 이건 살인 살인자 어디 영화 분 듯 이틀 차 안 물건 조작 차 안이 집 안이 활짝 아무나 문자 조작 비번 안 건 내용 자체 재밋 달팽이 빨 더 내용 전개 무난 아햏햏 아햏햏 아햏햏 단연 최고 진정 영화 최고 임 일이 기대

In [ ]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 534419
trigram 다섯개만 출력 : ['가가 각색' '가가 각색 선정' '가가 다반' '가가 다반 고흐' '가가 때문']


In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
pos_txt_embedding = model.encode([pos_txt])
candidate_embeddings = model.encode(candidates)

In [ ]:
top_n = 5
distances = cosine_similarity(pos_txt_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['우리나라 공포영화 낫다', '무지 만화 코스프레', '영화 정말 추리', '공포영화 달리 스릴', '만화책 정말 실사판']


In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(pos_txt_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)